# 🤖 Multi-Agent RAG: Client Billing Analyzer (Aiwyn Use Case)
This notebook focuses on analyzing client billing records, contracts, and terms using a multi-agent RAG system.

### 🎯 Goal
- Help CPA firms query and validate client billing terms
- Identify overcharges, late fees, and contract anomalies

**Agents Involved:**
- Document Ingestion Agent
- Indexer Agent (LlamaIndex + FAISS)
- Retrieval Agent (question answering)
- Billing Logic Agent (contract interpretation)
- GPT Explainer Agent (summary + risk flagging)
- Gradio UI for interaction

In [1]:
!pip uninstall -y llama-index

Found existing installation: llama-index 0.10.28
Uninstalling llama-index-0.10.28:
  Successfully uninstalled llama-index-0.10.28


In [2]:
# ✅ Install required libraries
# !pip install llama-index faiss-cpu openai langchain gradio pandas chromadb
# !pip install llama-index==0.10.28 faiss-cpu chromadb openai gradio pandas
# !pip install llama-index==0.10.28 chromadb faiss-cpu openai gradio pandas
# !pip install "llama-index[full]"==0.10.28 chromadb faiss-cpu openai gradio pandas
!pip install llama-index-vector-stores-chroma

  Using cached llama_index_core-0.12.42-py3-none-any.whl.metadata (2.4 kB)
Using cached llama_index_core-0.12.42-py3-none-any.whl (7.7 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-file 0.1.33 requires llama-index-core<0.11.0,>=0.10.37.post1, but you have llama-index-core 0.12.42 which is incompatible.
llama-index-embeddings-openai 0.1.11 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.42 which is incompatible.
llama-index-question-gen-openai 0.1.3 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.42 which is incompatible.
llama-index-readers-llama-parse 0.1.6 requir

## Restart Kernel

In [1]:
# 📦 Imports
import os
import gradio as gr
import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import openai

# 🔑 Set your API key
openai.api_key = os.getenv("OPENAI_API_KEY") or "your-api-key"

In [2]:
# 📂 Agent 1: Load documents (contracts, invoices, memos)
def ingest_billing_docs():
    return SimpleDirectoryReader("./billing_docs").load_data()

In [3]:
# 🧠 Agent 2: Build Vector Index
def build_billing_index(docs):
    context = ServiceContext.from_defaults(
        llm=OpenAI(model="gpt-3.5-turbo"),
        embed_model=OpenAIEmbedding()
    )
    index = VectorStoreIndex.from_documents(docs, service_context=context)
    index.storage_context.persist(persist_dir="./billing_storage")
    return index

In [4]:
# 🔎 Agent 3: Query documents
def query_billing_index(query):
    from llama_index.storage.storage_context import StorageContext
    storage_context = StorageContext.from_defaults(persist_dir="./billing_storage")
    index = VectorStoreIndex.load_from_storage(storage_context)
    return index.as_query_engine().query(query)

In [5]:
# 🎛️ Gradio UI: Ask questions about client billing terms
gr.Interface(
    fn=query_billing_index,
    inputs=gr.Textbox(label="Ask about billing or contracts", lines=2),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="📊 Client Billing Analyzer (RAG + GPT)",
    description="Load client invoices/contracts in ./billing_docs and query specific billing terms."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a0b8b09e513bb635c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
